# Stats Practice2

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats as stats
import sklearn as sk
import matplotlib.pyplot as plt

In [2]:
import sys
print(sys.version)

3.7.8 (tags/v3.7.8:4b47a5b6ba, Jun 28 2020, 08:53:46) [MSC v.1916 64 bit (AMD64)]


## 통계를 배우는 목적
1. 집단(모집단, 표본)에 대한 정량적 이해 – 분포, 대표값(점 추정, 구간 추정)
2. 의사 결정 – 검정 (표본의 모집단내 위치에 기반), 임계값, 유의 확률
3. 비교 – 표준값
4. 관련성(Relation) – 상관계수(Correlation), 상관비, 연관 계수(Association)
5. 예측 - 회귀

### [예제 1] '성별'과 '고백 방법'의 <u>연관계수</u>
고등학생 300명에게 '이성에게 어떤 방법으로 고백을 받고 싶은지' 조사하였다.  
출처 : Shin Takahashi, 만화로 쉽게 배우는 통계학, 2012, pp. 127-133

In [3]:
crosstab = pd.DataFrame([[34, 61, 53], [38, 40, 74]], \
                        index=['여', '남'], columns=['전화로', '메일로', '직접 만나서'])
crosstab

,전화로,메일로,직접 만나서
여,34,61,53
남,38,40,74


In [4]:
# 가설을 세운다
# H0 : 두 변수는 관계사 없다. (연관 계수 = 0)  
# H1 : 두 변수는 관계가 있다. (연관 계수 >0, 단측 검정)

# 독립성 검정 통계량 chi^2를 구한다
# chi^2 = sum((관측도수-기대도수)^2/기대도수)

# 표(교차 집계표)에 연령대 합계와 정당 합계를 추가한다
crosstab['성별 합계'] = crosstab.sum(axis=1)
crosstab.loc['고백 방법 합계'] = crosstab.sum(axis=0)

In [5]:
crosstab

,전화로,메일로,직접 만나서,성별 합계
여,34,61,53,148
남,38,40,74,152
고백 방법 합계,72,101,127,300


In [6]:
# 기대도수를 계산한다
exp_tab = crosstab.copy() # 깊은 복사
#exp_tab = exp_tab.astype('float') # 데이터 타입 변경

def calcExpFreq(crosstab, exp_tab):
    idx_col_sum = crosstab.shape[1] - 1
    idx_row_sum = crosstab.shape[0] - 1
    for irow in range(0, idx_row_sum):
        for icol in range(0, idx_col_sum):
            exp_tab.iat[irow, icol] = crosstab.iat[irow, idx_col_sum] * crosstab.iat[idx_row_sum, icol] \
                                        / crosstab.iat[idx_row_sum, idx_col_sum]

calcExpFreq(crosstab, exp_tab)

exp_tab

,전화로,메일로,직접 만나서,성별 합계
여,35.52,49.826667,62.653333,148
남,36.48,51.173333,64.346667,152
고백 방법 합계,72.00,101.000000,127.000000,300


In [9]:
# chi^2를 계산한다
# chi^2 = (관측도수-기대도수)^2/기대도수
chisq_tab = exp_tab.copy() # 깊은 복사

idx_col_sum = crosstab.shape[1] - 1
idx_row_sum = crosstab.shape[0] - 1
def calcChiSq(crosstab, exp_tab, chisq_tab):
    for irow in range(0, idx_row_sum):
        for icol in range(0, idx_col_sum):
            chisq_tab.iat[irow, icol] = (crosstab.iat[irow, icol] - exp_tab.iat[irow, icol])**2 \
                                        / exp_tab.iat[irow, icol]

calcChiSq(crosstab, exp_tab, chisq_tab)

# 표(교차 집계표)에 연령대 합계와 정당 합계를 다시 계산한다
chisq_tab['성별 합계'] = chisq_tab[['전화로', '메일로', '직접 만나서']].sum(axis=1)
chisq_tab.loc['고백 방법 합계'] = chisq_tab.iloc[0:idx_row_sum,:].sum(axis=0)

chisq_tab

,전화로,메일로,직접 만나서,성별 합계
여,0.065045,2.505553,1.487341,4.057939
남,0.063333,2.439618,1.448200,3.951151
고백 방법 합계,0.128378,4.945171,2.935541,8.009090


In [11]:
chisq_0 = chisq_tab.iat[idx_row_sum, idx_col_sum]

# 유의수준
alpha = 0.05 

# 확률변수(Random Variable)과 임계치를 구한다
rv_chi2 = stats.chi2(2) # 카이제곱분포의 자유도 = (행-1)*(열-1)
chisq_c = rv_chi2.ppf(1-alpha) # chi2.ppf (percent point function) : 분포의 좌측 누적 확률을 인자로 하여 임계치(기각역)를 구한다

# 유의 확률(p-value)도 구해 본다
p_value = 1 - rv_chi2.cdf(chisq_0)

print('Critical Vaule = ', chisq_c, ', p-value = ', p_value)

Critical Vaule =  5.991464547107979 , p-value =  0.01823258041007003


In [62]:
# chisq_0가 기각역에 포함되면 H0를 기각한다
# 그렇지 않으면 H0를 채택한다 (H0가 틀렸다고는 할 수 없다)
if (chisq_c <= chisq_0):
    print('chisq_c:', round(chisq_c,2), ' <= ', 'chisq_0:', round(chisq_0,2), '[H0 Rejected]')
else:
    print('chisq_c:', round(chisq_c,2), ' > ', 'chisq_0:', round(chisq_0,2), '[H0 Accepted]')

chisq_c: 5.99  <=  chisq_0: 8.01 [H0 Rejected]


In [63]:
# 결론 : 두 변수는 관계가 있다.

# 연관 계수를 구해 보자
# 연관 계수 V = sqrt(chisq_0^2 / N*min(행-1,열-1))
round(np.sqrt(chisq_0 / (300*1)),2)

0.16

In [65]:
# 독립성 검정 결과는 '두 변수가 관계가 있다'로 나왔는데 .. 
# 연관 계수는 0.16로 '매우 약하게 관련되어 있다(즉 관련이 없다)'로 나타나서
# 좀 의아한 생각이 든다 ..

### [예제 2] 패밀리 레스토랑의 '식사'와 '디저트'의 <u>연관계수</u>
패밀리 레스토랑에서 '20세 이상의 성인' 250명에게 조사하였다.  
출처 : Shin Takahashi, 만화로 쉽게 배우는 통계학, 2012, pp. 138-141

In [73]:
crosstab = pd.DataFrame([[43, 33], [51, 53], [29, 41]], \
                        index=['일식', '양식', '중식'], columns=['커피', '홍차'])
crosstab

,커피,홍차
일식,43,33
양식,51,53
중식,29,41


In [74]:
# 가설을 세운다
# H0 : 두 변수는 관계사 없다. (연관 계수 = 0)  
# H1 : 두 변수는 관계가 있다. (연관 계수 >0, 단측 검정)

# 독립성 검정 통계량 chi^2를 구한다
# chi^2 = sum((관측도수-기대도수)^2/기대도수)

# 표(교차 집계표)에 연령대 합계와 정당 합계를 추가한다
crosstab['식사 합계'] = crosstab.sum(axis=1)
crosstab.loc['디저트 합계'] = crosstab.sum(axis=0)

In [75]:
crosstab

,커피,홍차,식사 합계
일식,43,33,76
양식,51,53,104
중식,29,41,70
디저트 합계,123,127,250


In [76]:
# 기대도수를 계산한다
exp_tab = crosstab.copy() # 깊은 복사
#exp_tab = exp_tab.astype('float') # 데이터 타입 변경

def calcExpFreq(crosstab, exp_tab):
    idx_col_sum = crosstab.shape[1] - 1
    idx_row_sum = crosstab.shape[0] - 1
    for irow in range(0, idx_row_sum):
        for icol in range(0, idx_col_sum):
            exp_tab.iat[irow, icol] = crosstab.iat[irow, idx_col_sum] * crosstab.iat[idx_row_sum, icol] \
                                        / crosstab.iat[idx_row_sum, idx_col_sum]

calcExpFreq(crosstab, exp_tab)

exp_tab

,커피,홍차,식사 합계
일식,37.392,38.608,76
양식,51.168,52.832,104
중식,34.440,35.560,70
디저트 합계,123.000,127.000,250


In [77]:
# chi^2를 계산한다
# chi^2 = (관측도수-기대도수)^2/기대도수
chisq_tab = exp_tab.copy() # 깊은 복사

idx_col_sum = crosstab.shape[1] - 1
idx_row_sum = crosstab.shape[0] - 1
def calcChiSq(crosstab, exp_tab, chisq_tab):
    for irow in range(0, idx_row_sum):
        for icol in range(0, idx_col_sum):
            chisq_tab.iat[irow, icol] = (crosstab.iat[irow, icol] - exp_tab.iat[irow, icol])**2 \
                                        / exp_tab.iat[irow, icol]

calcChiSq(crosstab, exp_tab, chisq_tab)

# 표(교차 집계표)에 연령대 합계와 정당 합계를 다시 계산한다
chisq_tab['식사 합계'] = chisq_tab[['커피', '홍차']].sum(axis=1)
chisq_tab.loc['디저트 합계'] = chisq_tab.iloc[0:idx_row_sum,:].sum(axis=0)

chisq_tab

,커피,홍차,식사 합계
일식,0.841080,0.814589,1.655669
양식,0.000552,0.000534,0.001086
중식,0.859280,0.832216,1.691496
디저트 합계,1.700912,1.647340,3.348251


In [78]:
chisq_0 = chisq_tab.iat[idx_row_sum, idx_col_sum]

# 유의수준
alpha = 0.05 

# 확률변수(Random Variable)과 임계치를 구한다
rv_chi2 = stats.chi2(2) # 카이제곱분포의 자유도 = (행-1)*(열-1)
chisq_c = rv_chi2.ppf(1-alpha) # chi2.ppf (percent point function) : 분포의 좌측 누적 확률을 인자로 하여 임계치(기각역)를 구한다

chisq_c

5.991464547107979

In [79]:
# chisq_0가 기각역에 포함되면 H0를 기각한다
# 그렇지 않으면 H0를 채택한다 (H0가 틀렸다고는 할 수 없다)
if (chisq_c <= chisq_0):
    print('chisq_c:', round(chisq_c,2), ' <= ', 'chisq_0:', round(chisq_0,2), '[H0 Rejected]')
else:
    print('chisq_c:', round(chisq_c,2), ' > ', 'chisq_0:', round(chisq_0,2), '[H0 Accepted]')

chisq_c: 5.99  >  chisq_0: 3.35 [H0 Accepted]


In [80]:
# 결론 : 두 변수는 관계가 없다.

# 연관 계수를 구해 보자
# 연관 계수 V = sqrt(chisq_0^2 / N*min(행-1,열-1))
round(np.sqrt(chisq_0 / (300*1)),2)

0.11

In [65]:
# 이 예제에서는 독립성 검정 결과도 '두 변수가 관계가 없다'로 나오고
# 연관 계수도 0.11로 '매우 약하게 관련되어 있다(즉 관련이 없다)'로 나타나서
# 두 가지가 일치하고 있다